# Musicians- Medium

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app16-2")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@30980cc3

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@545a8cd7

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val band = hiveCxt.table("sqlzoo.band")
val composer = hiveCxt.table("sqlzoo.composer")
val composition = hiveCxt.table("sqlzoo.composition")
val concert = hiveCxt.table("sqlzoo.concert")
val has_composed = hiveCxt.table("sqlzoo.has_composed")
val musician = hiveCxt.table("sqlzoo.musician")
val performance = hiveCxt.table("sqlzoo.performance")
val performer = hiveCxt.table("sqlzoo.performer")
val place = hiveCxt.table("sqlzoo.place")
val plays_in = hiveCxt.table("sqlzoo.plays_in")

band: DataFrame = [band_no: int, band_name: string ... 4 more fields]
composer: DataFrame = [comp_no: int, comp_is: int ... 1 more field]
composition: DataFrame = [c_no: int, comp_date: string ... 2 more fields]
concert: DataFrame = [concert_no: int, concert_venue: string ... 3 more fields]
has_composed: DataFrame = [cmpr_no: int, cmpn_no: int]
musician: DataFrame = [m_no: int, m_name: string ... 4 more fields]
performance: DataFrame = [pfrmnc_no: int, gave: int ... 3 more fields]
performer: DataFrame = [perf_no: int, perf_is: int ... 2 more fields]
place: DataFrame = [place_no: int, place_town: string ... 1 more field]
plays_in: DataFrame = [player: int, band_id: int]

## 6.
**List the names, dates of birth and the instrument played of living musicians who play a instrument which Theo also plays.**

In [5]:
(musician.filter((musician("died").isNull) && 
                 (! musician("m_name").like("Theo%")))
 .join(performer, (musician("m_no") === col("perf_is")))
 .join(musician.filter(musician("m_name").like("Theo%"))
        .join(performer, (musician("m_no") === performer("perf_is")))
        .select("instrument"), "instrument")
 .select("m_name", "born", "instrument")
 .orderBy("m_name")
 .showHTML())

m_name,born,instrument
Harry Forte,1951-02-28,drums
Harry Forte,1951-02-28,violin
James First,1965-06-10,violin
Jeff Dawn,1945-12-12,violin
John Smith,1950-03-03,violin


## 7.
**List the name and the number of players for the band whose number of players is greater than the average number of players in each band.**

In [6]:
(band.join(plays_in, (band("band_no") === plays_in("band_id")))
 .join(performer, (plays_in("player") === performer("perf_no")))
 .select("band_name", "perf_is")
 .distinct()
 .groupBy("band_name")
 .agg(count("perf_is").alias("n_mbr"))
 .withColumn("avg_mbr", avg("n_mbr").over(Window.partitionBy(lit(0))))
 .filter(col("n_mbr") > col("avg_mbr"))
 .select("band_name", "n_mbr")
 .showHTML())

band_name,n_mbr
ROP,7
AASO,7
Oh well,5


## 8.
**List the names of musicians who both conduct and compose and live in Britain.**

In [7]:
(musician.join(composer, (musician("m_no") === composer("comp_is")))
 .join(place.filter(place("place_country").isin(List("England", "Scotland"): _*)), 
       (musician("living_in") === place("place_no")))
 .join(performance, (musician("m_no") === performance("conducted_by")))
 .select("m_name")
 .distinct()
 .orderBy("m_name")
 .showHTML())

m_name
Fred Bloggs
Jeff Dawn
Phil Hot
Rose Spring
Tony Smythe


## 9.
**Show the least commonly played instrument and the number of musicians who play it.**

In [8]:
(performer.join(plays_in, (performer("perf_no") === plays_in("player")))
 .join(performance, (plays_in("band_id") === performance("gave")))
 .groupBy("instrument")
 .agg(count("perf_no").alias("n_musician"))
 .orderBy("n_musician")
 .limit(1)
 .showHTML())

instrument,n_musician
clarinet,1


## 10.
**List the bands that have played music composed by Sue Little; Give the titles of the composition in each case.**

In [9]:
(composition
 .join(has_composed, (composition("c_no") === has_composed("cmpn_no")))
 .join(composer, (has_composed("cmpr_no") === composer("comp_no")))
 .join(musician.filter(musician("m_name") === "Sue Little"),
       (composer("comp_is") === musician("m_no")))
 .select("c_no", "c_title")
 .join(performance, (col("c_no") === performance("performed")))
 .join(band, (performance("gave") === band("band_no")))
 .select("band_name", "c_title")
 .orderBy("band_name")
 .showHTML())

band_name,c_title
BBSO,Slow Song
BBSO,Slow Symphony Blowing
Somebody Loves this,Slow Symphony Blowing
Swinging strings,Slow Song
The left Overs,Slow Song


In [10]:
spark.stop()